### Evaluation of Variational Autoencoder (VAE) for KDD CUP

## The Method

1. Prepare data & Partition the data into 80-20 train-test split
2. Define Models & Parameters
3. Perform Cross-Validation on the training data
3. (Re)Train the models on the training data, either vanilla model or with best hyperparameters
4. Evaluate the (re)trained models on the test data
5. Final evaluation of the models

![The Method](method.png)

Source: [scikit-learn.org - Cross-validation](https://scikit-learn.org/1.5/modules/cross_validation.html#cross-validation-and-model-selection)

In [20]:
# Import libraries
import pandas as pd
import numpy as np
from pandas import DataFrame as df
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_validate
from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline as imblearn_pipeline # Use imblearn's pipeline
from sklearn.metrics import make_scorer, classification_report, precision_score, recall_score, f1_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.base import  BaseEstimator,ClassifierMixin

# Import ML libraries
import keras
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from scikeras.wrappers import KerasClassifier
import keras_tuner
from keras import layers
from keras import ops
import tensorflow as tf

from pprint import pprint
from time import time
import joblib

import warnings
warnings.filterwarnings('ignore')

RANDOM_STATE = 42
CROSS_VAL_SPLITS = 5
CKP_PREFIX = 'KDD_'

np.random.seed(RANDOM_STATE)

# pandas options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


### 1. Prepare data & Partition the data into 80-20 train-test split

In [ ]:
# Importing the dataset
path_to_data = '../data/kddcup/kddcup_data_corrected.csv'

# col names from https://kdd.ics.uci.edu/databases/kddcup99/kddcup.names
col_names = [
    "duration",
    "protocol_type",
    "service",
    "flag",
    "src_bytes",
    "dst_bytes",
    "land",
    "wrong_fragment",
    "urgent",
    "hot",
    "num_failed_logins",
    "logged_in",
    "num_compromised",
    "root_shell",
    "su_attempted",
    "num_root",
    "num_file_creations",
    "num_shells",
    "num_access_files",
    "num_outbound_cmds",
    "is_host_login",
    "is_guest_login",
    "count",
    "srv_count",
    "serror_rate",
    "srv_serror_rate",
    "rerror_rate",
    "srv_rerror_rate",
    "same_srv_rate",
    "diff_srv_rate",
    "srv_diff_host_rate",
    "dst_host_count",
    "dst_host_srv_count",
    "dst_host_same_srv_rate",
    "dst_host_diff_srv_rate",
    "dst_host_same_src_port_rate",
    "dst_host_srv_diff_host_rate",
    "dst_host_serror_rate",
    "dst_host_srv_serror_rate",
    "dst_host_rerror_rate",
    "dst_host_srv_rerror_rate",
    "label",
]
# from https://kdd.ics.uci.edu/databases/kddcup99/kddcup.names
categorical_cols = [
    "protocol_type",
    "service",
    "flag",
    "land",
    "logged_in",
    "is_host_login",
    "is_guest_login",
]

# Read data (10 % subset of the original dataset)
data = pd.read_csv(path_to_data, names=col_names, header=None)

# Label encoding
data["label"] = data["label"].apply(lambda x: "attack" if x != "normal." else 'normal')
le = LabelEncoder()
y = le.fit_transform(data["label"])
X = data.drop("label", axis=1)

# Extract label mapping
label_mapping = {label: index for index, label in enumerate(le.classes_)}
normal_label = label_mapping['normal']
attack_label = label_mapping['attack']
print("Label Mapping:", label_mapping)
print("Normal Label:", normal_label)
print("Attack Label:", attack_label)

# Visualize class imbalance
plt.bar(["Attack", "Normal"], data["label"].value_counts())
total = len(data)
fraud_percentage = (data["label"].value_counts()[1] / total) * 100
normal_percentage = (data["label"].value_counts()[0] / total) * 100

plt.text(0, data["label"].value_counts()[0] / 2, f'{normal_percentage:.2f}%', ha='center', color='black')
plt.text(1, data["label"].value_counts()[1] / 2, f'{fraud_percentage:.2f}%', ha='center', color='black')
plt.title("Class Distribution")
plt.show()

# create training and test partitions with 80-20 split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y)

# Fit OneHotEncoder to learn categories from the training data. This is necessary to ensure that the same categories are used for random splits during cross-validation
one_hot_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False, dtype="int8")
one_hot_encoder.fit(X_train[categorical_cols])
learned_categories = one_hot_encoder.categories_

# Define numerical columns & indices
num_cols = data.drop(categorical_cols, axis=1)
num_indices = [data.columns.get_loc(col) for col in num_cols.drop('label', axis=1).columns]
cat_indices = [data.columns.get_loc(col) for col in categorical_cols]

# Set class weights
class_weight = {label[0]: 1.0 / count for label, count in df(y_train).value_counts().items()}
print("Weights: ", class_weight)

# Define preprocessing transformers and pipeline

# Numerical transformer. Scales numerical data using StandardScaler
numeric_transformer = imblearn_pipeline(
    steps=[("scaler", StandardScaler())], verbose=True
)
numeric_transformer.set_output(transform="pandas")

# Categorical transformer. Encodes categorical data using OneHotEncoder
categorical_transformer = imblearn_pipeline(
    steps=[
        (
            "onehot",
            OneHotEncoder(
                # handle_unknown="ignore",
                sparse_output=False,
                categories=learned_categories,
                dtype="int8",
            ),
        )
    ],
    verbose=True,
)
categorical_transformer.set_output(transform="pandas")

# Define preprocessor with ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_indices),
        ("cat", categorical_transformer, cat_indices),
    ],
    verbose=True,
)


# Check preprocessing pipeline and get target shape
transformed_sample = preprocessor.fit_transform(X_train)
print(f"Transformed X_train shape: {transformed_sample.shape}")

### 2. Define models and parameters

In [22]:
# VAE Keras Custom Model Class 

# Code based on Bank et al (2021)
@keras.saving.register_keras_serializable()
class VAE(keras.Model):
    def __init__(self, encoder, decoder, kl_beta=1, **kwargs):
        super().__init__(**kwargs)
        self.decoder = decoder
        self.encoder = encoder
        self.loss_tracker =  keras.metrics.Mean(name="loss")
        self.seed_generator = keras.random.SeedGenerator(RANDOM_STATE)
        self.kl_beta = kl_beta # weight to put more or less balance on reconstruction loss
        
    def compile(self, optimizer):
        super().compile()
        self.optimizer = optimizer
    
    # @tf.function
    def reconstruction_loss(self, x, f_z):
        #reconstruction_loss = keras.losses.binary_crossentropy(x, f_z)
        mse = keras.losses.MeanSquaredError()
        reconstruction_loss = mse(x, f_z)

        return reconstruction_loss
    
    # @tf.function
    def loss_fn(self, x, f_z, z_mean, z_log_var):
        # Ensure reduction across the batch dimension
        reconstruction_loss = tf.reduce_mean(self.reconstruction_loss(x, f_z))
        
        # KL divergence calculation
        kl_loss = -0.5 * tf.reduce_sum(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var + 1e-8), axis=1)
        kl_loss = tf.reduce_mean(kl_loss)
        
        # total loss calculation
        total_loss = reconstruction_loss + (self.kl_beta * kl_loss)
        return total_loss, reconstruction_loss, kl_loss
    
    @staticmethod # Define the encoder model. See Bank et al (2021, p 8)
    def get_encoder(data_dim, latent_dim, hidden_dims, activation=keras.layers.ReLU(), dropout_rate=0): 
        """ Encoder Net
        
        Takes input x and maps it into a mean and covariance that determine the approximate posterior distribution of the latent space.

        Args:
            data_dim (_type_): Input dimension. Usually equal to n features.
            latent_dim (_type_): Size of the bottleneck.
            hidden_dims: List of hidden dims, e.g., [20, 10, 8]
            activtion: Activtion function for hidden dims

        Returns:
            z_mean: Mean
            z_log_var: Covariance
        """
        # Input Layer
        inputs = layers.Input(shape=(data_dim,))
        
        # Hidden Layer for compression
        x = inputs
        for dim in hidden_dims:
            x = layers.Dense(dim)(x)
            x = activation(x)
            if dropout_rate != 0:
                x = layers.Dropout(dropout_rate)(x)
              
        # calculate mean g(x) and covariance h(x) of the distribution
        z_mean = layers.Dense(latent_dim)(x)
        z_log_var = layers.Dense(latent_dim)(x)

        return keras.Model(inputs, [z_mean, z_log_var], name='encoder')
    
    @staticmethod # Define the decoder model f(z)
    def get_decoder(latent_dim, output_dim, hidden_dims, activation=keras.layers.ReLU(), dropout_rate=0):
        # take sample from latent space as input
        latent_inputs = layers.Input(shape=(latent_dim,))
        
        # Hidden Layer for reconstruction
        x = latent_inputs
        for dim in reversed(hidden_dims):
            x = layers.Dense(dim)(x)
            x = activation(x)
            if dropout_rate != 0:
                x = layers.Dropout(dropout_rate)(x)
        
        # output reconstruction
        outputs = layers.Dense(output_dim, activation='sigmoid')(x)
        return keras.Model(latent_inputs, outputs, name='decoder')
    
    def sampling(self, args): # Define the sampling function. See Bank et al (2021, p 8)
        """ Sample from latent space by applying 'Reparameterization'

        Args:
            args (list): List of mean and covariance (z_mean, z_log_var)

        Returns:
            z: Sample from the latent space
        """
        # unpack mean g(x) and covariance h(x)
        z_mean, z_log_var = args
        
        # epsilon is a normal distribution: N(0, I)
        epsilon = keras.random.normal(shape=tf.shape(z_mean), seed=self.seed_generator)
        
        # Sampling via reparametrisation trick: z = h(x)  * espilon + g(x)
        return epsilon * tf.exp(z_log_var * .5) + z_mean

    # @tf.function
    def train_step(self, data):
        # data = random minibatch of M datapoints
        x = data
        
        with tf.GradientTape() as tape:
            # encode input to mean g(x) and covariance h(x)
            z_mean, z_log_var = self.encoder(x)
            
            # sample random z from latent space
            z = self.sampling((z_mean, z_log_var))
            
            # decode z via f(z)
            f_z = self.decoder(z)

            # calculate reconstruction loss
            loss, reconstruction_loss, kl_loss = self.loss_fn(x, f_z, z_mean, z_log_var)
            
        grads = tape.gradient(loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        
        self.loss_tracker.update_state(loss)         
        
        return {
            "loss": self.loss_tracker.result(),
            "reconstruction_loss": reconstruction_loss,
            "kl_loss": kl_loss
        }
        
    def test_step(self, data):
        x = data
        z_mean, z_log_var = self.encoder(x)
        
        z = self.sampling((z_mean, z_log_var))
        f_z = self.decoder(z)
        
        _, reconstruction_loss, _ = self.loss_fn(x, f_z, z_mean, z_log_var)
        return {
            "reconstruction_loss": reconstruction_loss,
        }

    # def call(self, data):
    #     x = data
    #     z_mean, z_log_var = self.encoder(x)
        
    #     z = self.sampling((z_mean, z_log_var))
    #     f_z = self.decoder(z)
            
    #     return self.reconstruction_loss(x, f_z)

    def call(self, data):
        x = data
        z_mean, z_log_var = self.encoder(x)
        z = self.sampling((z_mean, z_log_var))
        f_z = self.decoder(z)
        return f_z  # Return reconstructed outputs

In [30]:
class VAEClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, data_dim, latent_dim=80, hidden_dims=[128, 128, 64], 
                 kl_beta=0.5, dropout_rate=0.0, batch_size=128, epochs=100, learning_rate=1e-6, cb_early_stopping_patience=5, normal_label=1):
        self.data_dim = data_dim
        self.latent_dim = latent_dim
        self.hidden_dims = hidden_dims
        self.kl_beta = kl_beta
        self.dropout_rate = dropout_rate
        self.batch_size = batch_size
        self.epochs = epochs
        self.vae = None
        self.activation = keras.layers.ReLU()
        self.learning_rate = learning_rate
        self.cb_early_stopping_patience = cb_early_stopping_patience
        self.normal_label = normal_label

        print("type of learning rate: ", type(self.learning_rate))
    def _build_model(self):

        enc = VAE.get_encoder(self.data_dim, self.latent_dim, self.hidden_dims, 
                              self.activation, self.dropout_rate)
        dec = VAE.get_decoder(self.latent_dim, self.data_dim, self.hidden_dims, 
                              self.activation, self.dropout_rate)
        vae = VAE(enc, dec, self.kl_beta)

        optimizer = keras.optimizers.Adam(learning_rate=self.learning_rate)
        vae.compile(optimizer=optimizer)
        return vae
    
    def fit(self, X, y):
        X_train = X[y == self.normal_label]  # Only train on normal samples
        X_train = np.array(X_train, dtype=np.float32)
        dataset = tf.data.Dataset.from_tensor_slices(X_train).batch(self.batch_size)
        self.vae = self._build_model()
        self.vae.fit(dataset, epochs=self.epochs, verbose=1, callbacks=[
            keras.callbacks.EarlyStopping(
                patience=self.cb_early_stopping_patience, monitor='reconstruction_loss', mode='min'
            )
        ])
        
        # Compute reconstruction losses on the training data
        reconstructed_X = self.vae.predict(X_train)
        self.training_losses = np.mean(np.square(X_train - reconstructed_X), axis=1)
        
        return self
    
    def predict(self, X):
        X = np.array(X, dtype=np.float32)
        reconstructed_X = self.vae.predict(X)
        
        # Compute per-sample reconstruction losses
        losses = np.mean(np.square(X - reconstructed_X), axis=1)
        
        # Determine the threshold based on training losses
        threshold = np.percentile(self.training_losses, 95)  # Adjust percentile if needed
        predictions = (losses > threshold).astype(int)  # 1 indicates anomaly (attack)
        
        print(f"Threshold used: {threshold}")
        print(f"Predictions unique values: {np.unique(predictions, return_counts=True)}")
        
        return predictions

In [ ]:
vae_estimator = VAEClassifier(data_dim=transformed_sample.shape[1], normal_label=normal_label)

# Define pipeline
vae_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', vae_estimator)
])

# Define scoring metrics
scoring = {
    'f1': make_scorer(f1_score, zero_division=0),
    'precision': make_scorer(precision_score, zero_division=0),
    'recall': make_scorer(recall_score, zero_division=0)
}

# test the model
vae_pipeline.fit(X_train[y_train == normal_label], y_train[y_train == normal_label])
y_pred = vae_pipeline.predict(X_test)

In [ ]:
y_pred = vae_pipeline.predict(X_test)

In [ ]:
test_transformed = preprocessor.transform(X_test[y_test == normal_label])
print("Test transformed shape: ", test_transformed.shape)

# Compute reconstruction losses manually
reconstructed_X = vae_pipeline.named_steps['classifier'].vae.predict(test_transformed)

In [ ]:
# evaluate f1
f1 = f1_score(y_test, y_pred, zero_division=0)
precision = precision_score(y_test, y_pred, zero_division=0)
recall = recall_score(y_test, y_pred, zero_division=0)

print(f"F1 Score: {f1}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")

In [ ]:
# Preprocess data
X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)

# Ensure the data is in the correct format
X_train_transformed = np.array(X_train_transformed, dtype=np.float32)
X_test_transformed = np.array(X_test_transformed, dtype=np.float32)

# Initialize the VAEClassifier
vae_classifier = VAEClassifier(
    data_dim=X_train_transformed.shape[1],  # Adjust based on your input shape
    latent_dim=80,  # Modify latent_dim based on your requirement
    hidden_dims=[128, 128, 64],  # Adjust hidden layer dimensions if necessary
    kl_beta=0.5,
    dropout_rate=0.0,
    batch_size=128,
    epochs=10,  # Set to a lower value for quick testing; adjust as needed
    learning_rate=1e-3,  # Adjust as per your need
    cb_early_stopping_patience=5,
    normal_label=normal_label  # Make sure 'normal_label' is correctly defined
)

# Fit the model
print("Starting model training...")
vae_classifier.fit(X_train_transformed, y_train)

# Predict on the test data
print("Making predictions...")
predictions = vae_classifier.predict(X_test_transformed)

# Evaluate F1 Score
print("Evaluating F1 score...")
f1 = f1_score(y_test, predictions, average='binary')  # Use 'binary' or 'macro' depending on your target setup
print(f"F1 Score: {f1:.2f}")

In [ ]:

# Perform cross-validation
skf = StratifiedKFold(n_splits=2)
results = cross_validate(vae_pipeline, X_train[y_train == normal_label], y_train[y_train == normal_label], cv=skf, scoring=scoring, return_train_score=False)

# Print results
for metric in scoring.keys():
    mean_score = np.mean(results[f'test_{metric}'])
    std_score = np.std(results[f'test_{metric}'])
    print(f"{metric.capitalize()} - Mean: {mean_score:.4f}, Std: {std_score:.4f}")

In [33]:
# Define pipeline
vae_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', VAEClassifier(data_dim=transformed_sample.shape[1], normal_label=normal_label))
])

# Filter X_train and y_train based on normal label
X_train_filtered = X_train[y_train == normal_label]
y_train_filtered = y_train[y_train == normal_label]

# Ensure X_train_filtered and y_train_filtered are converted to numpy arrays for proper indexing
X_train_filtered = np.array(X_train_filtered)
y_train_filtered = np.array(y_train_filtered)

# Initialize lists to store metrics for each CV run
precision_scores = []
recall_scores = []
f1_scores = []

# Cross-validation with StratifiedKFold
for i, (train_idx, test_idx) in enumerate(StratifiedKFold(n_splits=2).split(X_train_filtered, y_train_filtered)):
    print(f"[CV {i+1} ...]")

    # Split data
    X_train_split, X_test_split = X_train_filtered[train_idx], X_train_filtered[test_idx]
    y_train_split, y_test_split = y_train_filtered[train_idx], y_train_filtered[test_idx]

    # Fit the pipeline on the normal samples from the training split
    vae_pipeline.fit(X_train_split[y_train_split == normal_label], y_train_split[y_train_split == normal_label])

    # Predict on the test set
    y_pred = vae_pipeline.predict(X_test_split)

    print(f"type of y_pred: {type(y_pred)}")
    if not isinstance(y_pred, np.ndarray):
        y_pred = np.array(y_pred)

    # Ensure y_pred has the same length as y_test_split for metric calculation
    if len(y_pred) != len(y_test_split):
        raise ValueError(f"Inconsistent prediction length: {len(y_pred)} vs {len(y_test_split)}")

    # Calculate metrics
    precision = precision_score(y_test_split, y_pred, pos_label=attack_label)  # Specify pos_label as needed
    recall = recall_score(y_test_split, y_pred, pos_label=attack_label)
    f1 = f1_score(y_test_split, y_pred, pos_label=attack_label)

    # Append metrics for this CV run
    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)

# Calculate mean and standard deviation for each metric
precision_mean = np.mean(precision_scores)
precision_std = np.std(precision_scores)

recall_mean = np.mean(recall_scores)
recall_std = np.std(recall_scores)

f1_mean = np.mean(f1_scores)
f1_std = np.std(f1_scores)

# Print the results
print(f"Precision: {precision_mean:.2f} ± {precision_std:.2f}")
print(f"Recall: {recall_mean:.2f} ± {recall_std:.2f}")
print(f"F1-Score: {f1_mean:.2f} ± {f1_std:.2f}")

type of learning rate:  <class 'float'>
[CV 1 ...]
[Pipeline] ............ (step 1 of 1) Processing scaler, total=   0.0s
[ColumnTransformer] ........... (1 of 2) Processing num, total=   0.0s
[Pipeline] ............ (step 1 of 1) Processing onehot, total=   0.0s
[ColumnTransformer] ........... (2 of 2) Processing cat, total=   0.0s
Epoch 1/100
190/190 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - kl_loss: 806.7906 - loss: 305.2220 - reconstruction_loss: 0.4954
Epoch 2/100
190/190 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - kl_loss: 765.0170 - loss: 289.5061 - reconstruction_loss: 0.4938
Epoch 3/100
190/190 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - kl_loss: 719.7486 - loss: 272.4652 - reconstruction_loss: 0.4920
Epoch 4/100
190/190 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - kl_loss: 676.7652 - loss: 256.2779 - reconstruction_loss: 0.4902
Epoch 5/100
190/190 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - kl_loss: 636.9020 - loss: 241.2633 - reconstruction_loss: 0.4887
Epoch 6/100
190/190 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - kl_loss: 6